In [1]:
def listReshape(lst, dim):
  '''
  listReshape takes in a 1d list and an integer value for the dimension

  returns: a 2d list with shape dim ** 2
  '''
  if len(lst) != dim ** 2:
    print("DIMENSIONALITY ERROR")
    raise ValueError
  return [[lst[j * i + j] for j in range(dim)] for i in range(dim)]


def convoluteSubArray(matrix, kernel, reach, ind):
  '''
  convoluteSubArray takes in the matrix to be convoluted, the kernel doing the convolution, the reach of the kernel size to any side from the ind location (int, int)

  returns: a single value that corresponds to the convolution of the kernel and the submatrix of the original matrix
  '''
  row, col = ind[0], ind[1]
  subMatrix = matrix[row - reach: row + reach + 1, col - reach: col + reach + 1]
  rang = len(subMatrix[0])
  convoluteValue = 0
  for col in range(rang):
    for row in range(rang):
      convoluteValue += (subMatrix[col][row] * kernel[col][row])
  return convoluteValue

def convolutionHelper(matrix, kernel_size, kernel, input_dim):
  '''
  convolutionHelper takes in the matrix to be convoluted, the kernel size, the kernel, and the dimensionality of the matrix to be convoluted

  returns: convoluted matrix with dimensionality (input_dim - kernel_size) ** 2
  '''
  returnMatrixDim = (input_dim - kernel_size)
  returnMatrix = []

  startInd = (kernel_size - 1) // 2

  for col in range(startInd, input_dim - startInd):
    for row in range(startInd, input_dim - startInd):
      returnMatrix.append(max(0, convoluteSubArray(matrix, kernel, startInd, (row, col)))) # this part of the code applies ReLu activation function by taking max of 0 and convolution
  # Reshaping the matrix
  returnMatrix = listReshape(returnMatrix, (input_dim - kernel_size + 1))
  return returnMatrix

def Convolution2D(matrix, kernel_size, kernels, input_dim):
  '''
  This function takes in the matrix (image to be convoluted), kernel_size (integer value | note: only accepts square matrices), kernels (the kernels to be used), and the input_dim (int)

  returns: a matrix for each kernel with dimensionality (input_dim - kernel_size) ** 2 -> dimensionality: (# kernels, (input_dim - kernel_size) ** 2)
  '''
  returnMatrices = []

  for kernel in range(len(kernels)):
    returnMatrices.append(convolutionHelper(matrix, kernel_size, kernels[kernel], input_dim))
  return returnMatrices

In [2]:
def getMaxValue(subMatrix, dim):
  '''
  getMaxValue takes in a submatrix with dimensionality of dim

  returns: the maximum value within the sub matrix
  '''
  maxValue = 0
  for col in range(0, dim):
    for row in range(0, dim):
      maxValue = max(maxValue, subMatrix[col, row])
  return maxValue

def MaxPool2D(matrix, pooling_size, input_dim, stride):
  '''
  MaxPool2D takes in the matrix to be pooled, the size of the pooling range, the size of the matrix to be pooled, and stride

  returns: a maxpooled matrix with dimensionality ((input_dim - pooling_size) / stride) + 1
  '''
  returnMatrix = []
  for col in range(0, input_dim - (pooling_size // 2) + 1, stride):
    for row in range(0, input_dim - (pooling_size // 2) + 1, stride):
      subMatrix = matrix[col: col + (pooling_size // 2) + 1, row: row + (pooling_size // 2) + 1]
      returnMatrix.append(getMaxValue(subMatrix, pooling_size))
  returnMatrix = listReshape(returnMatrix, ((input_dim - pooling_size) // 2) + 1)
  return returnMatrix

In [4]:
# Setting up our model
class CNN():
  def __init__(self, convolutionKernels, kernelSize, poolingSize, stride):
    self.ConvKernels = convolutionKernels
    self.KernelDim = kernelSize
    self.poolDim = poolingSize
    self.poolStide = stride
    self.modelLayers = []

  def addLayer(self, layerType, input_dim):
    self.modelLayers.append((layerType, input_dim))

  def forward(self, inputData):
    x = inputData
    for i in range(len(self.modelLayers)):
      if self.modelLayers[i][0] == "Convolution":
        x = Convolution2D(x[j] for j in range(len(x)))
      x = self.modelLayers[i](x)
1
kernels = [
    # Left vertical edge detection
    [[-1, 0, 1],
     [-1, 0, 1],
     [-1, 0, 1]],
    # Right vertical edge detection
    [[1, 0, -1],
     [1, 0, -1],
     [1, 0, -1]],
    # Top horizontal edge detection
    [[1, 1, 1],
     [0, 0, 0],
     [-1, -1, -1]],
    # Bottom horizontal edge detection
    [[-1, -1, -1],
     [0, 0, 0],
     [1, 1, 1]],
    # Embossing kernel
    [[-2, -1, 0],
     [-1, 1, 1],
     [0, 1, 2]]
]

In [5]:
model = CNN(kernels, 3, 2, 2)
model.addLayer("Convolution", 8)
model.addLayer("Pooling", 6)
model.addLayer("Convolution", 3)
# Brain Hurt, how to put into a class